<a href="https://colab.research.google.com/github/UserMarcos/03MIAR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2)_heuristico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Marcos Castro Castro  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

## Problema 1. Organizar sesiones de doblaje(I)
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
- Número de actores: 10
- Número de tomas : 30
- Actores/Tomas : https://bit.ly/36D8IuK
  - 1 indica que el actor participa en la toma
  - 0 en caso contrario

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [46]:
# Creamos la tabla de tomas
Numero_actores = 10

Actores_por_Toma = [
    {1, 2, 3, 4, 5},  # Toma 1
    {3, 4, 5},        # Toma 2
    {2, 5, 7},        # Toma 3
    {1, 2, 7, 8},     # Toma 4
    {2, 4, 8},        # Toma 5
    {1, 2, 4, 5},     # Toma 6
    {1, 2, 4, 5},     # Toma 7
    {1, 2, 6},        # Toma 8
    {1, 2, 4},        # Toma 9
    {1, 2, 6, 9},     # Toma 10
    {1, 2, 3, 5, 8},  # Toma 11
    {1, 2, 3, 4, 6},  # Toma 12
    {1, 4, 5},        # Toam 13
    {1, 3, 6},        # Toma 14
    {1, 2, 7},        # Toma 15
    {4, 10},          # Toma 16
    {1, 3},           # Toma 17
    {3, 6},           # Toma 18
    {1, 3},           # Toma 19
    {1, 3, 4, 5},     # Toma 20
    {6, 8},           # Toma 21
    {1, 2, 3, 4},     # Toma 22
    {1, 3},           # Toma 23
    {3, 6},           # Toma 24
    {1, 2, 4, 10},    # Toma 25
    {1, 3, 5, 9},
    {4, 5},
    {1, 4},
    {1, 5, 6},
    {1, 4}
    ]

Actores_por_Toma

[{1, 2, 3, 4, 5},
 {3, 4, 5},
 {2, 5, 7},
 {1, 2, 7, 8},
 {2, 4, 8},
 {1, 2, 4, 5},
 {1, 2, 4, 5},
 {1, 2, 6},
 {1, 2, 4},
 {1, 2, 6, 9},
 {1, 2, 3, 5, 8},
 {1, 2, 3, 4, 6},
 {1, 4, 5},
 {1, 3, 6},
 {1, 2, 7},
 {4, 10},
 {1, 3},
 {3, 6},
 {1, 3},
 {1, 3, 4, 5},
 {6, 8},
 {1, 2, 3, 4},
 {1, 3},
 {3, 6},
 {1, 2, 4, 10},
 {1, 3, 5, 9},
 {4, 5},
 {1, 4},
 {1, 5, 6},
 {1, 4}]

In [48]:
N_Actores_por_toma = [len(x) for x in Actores_por_Toma]
print("Número de actores por toma: ", N_Actores_por_toma)

N_Tomas_por_actor = []
for i in range(Numero_actores):
  Numero_tomas = 0
  for Toma in Actores_por_Toma:
    if i+1 in Toma:
      Numero_tomas = Numero_tomas + 1
  N_Tomas_por_actor.append(Numero_tomas)

print("Número de tomas por actor: ", N_Tomas_por_actor)

Número de actores por toma:  [5, 3, 3, 4, 3, 4, 4, 3, 3, 4, 5, 5, 3, 3, 3, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 4, 2, 2, 3, 2]
Número de tomas por actor:  [22, 14, 13, 15, 11, 8, 3, 4, 2, 2]


In [43]:
def Get_Actores_Dia(Dias_Toma, Actores_Toma):
  Actores_Dia = dict()
  for i in range(len(Dias_Toma)):
    N_Dia = Dias_Toma[i]
    if  N_Dia in Actores_Dia:
      Actores_Dia[N_Dia].update(Actores_Toma[i])
    else:
      Actores_Dia[N_Dia] = Actores_Toma[i].copy()
  return Actores_Dia

Dia_por_Toma = [0, 1, 0, 2, 2]
print(Dia_por_Toma)
print(Actores_por_Toma)
Actores_Dia = Get_Actores_Dia(Dia_por_Toma, Actores_por_Toma)
Actores_Dia

[0, 1, 0, 2, 2]
[{1, 2, 3, 4, 5}, {3, 4, 5}, {2, 5, 7}, {8, 1, 2, 7}, {8, 2, 4}, {1, 2, 4, 5}, {1, 2, 4, 5}, {1, 2, 6}, {1, 2, 4}, {1, 2, 6, 9}, {1, 2, 3, 5, 8}, {1, 2, 3, 4, 6}, {1, 4, 5}, {1, 3, 6}, {1, 2, 7}, {10, 4}, {1, 3}, {3, 6}, {1, 3}, {1, 3, 4, 5}, {8, 6}, {1, 2, 3, 4}, {1, 3}, {3, 6}, {1, 2, 10, 4}, {1, 3, 5, 9}, {4, 5}, {1, 4}, {1, 5, 6}, {1, 4}]


{0: {1, 2, 3, 4, 5, 7}, 1: {3, 4, 5}, 2: {1, 2, 4, 7, 8}}

In [44]:
def Get_Coste(Actores_Dia):
  Coste = 0
  for Dia in Actores_Dia:
    Coste += len(Actores_Dia[Dia])
  return Coste

def Get_Coste_Inf(Dias_Toma, Actores_Toma):
  Valor = Get_Coste(Get_Actores_Dia(Dias_Toma, Actores_Toma))

  # Estimación
  for i in range(Numero_actores):
    Numero_tomas = 0
    for j in range(len(Dias_Toma), len(Actores_Toma)):
      if i+1 in Actores_Toma[j]:
        Numero_tomas += 1
    Valor += (Numero_tomas - len(Dias_Toma))//6
  return Valor

def Get_Coste_Sup(Dias_Toma, Actores_Toma):
  Valor = Get_Coste(Get_Actores_Dia(Dias_Toma, Actores_Toma))

  # Estimación
  for i in range(len(Dias_Toma), len(Actores_Toma)):
    Valor += len(Actores_Toma)
  return Valor

print(Get_Coste(Actores_Dia))

print(Get_Coste_Sup(Dia_por_Toma, Actores_por_Toma))

14
764


In [ ]:
def Crear_Hijos(Nodo):
  Hijos = []
  long = len(Nodo)
  for i in range(long):
    n = Nodo.count(i)
    if n < 6 and n > 0:
      Hijos.append({'s':Nodo +(i,)})
  Hijos.append({'s':Nodo +(long,)})
  return Hijos

Crear_Hijos((0, 0, 2))

[{'s': (0, 0, 2, 0)}, {'s': (0, 0, 2, 2)}, {'s': (0, 0, 2, 3)}]

In [ ]:
def Calcular_Sesiones(Dias_Toma, Actores_Toma):
  if (len(Dias_Toma) >= len(Actores_Toma)):
    return Get_Gasto(Get_Actores_Dia(Dias_Toma, Actores_Toma))
  else:
    for i in range(len(Dias_Toma)):


    Dias_Toma.append(len(Dias_Toma))
    return Calcular_Sesiones(Dias_Toma, Actores_Toma)

Dias_por_Toma = []
n = Calcular_Sesiones(Dias_por_Toma, Actores_por_Toma)
print(Dias_por_Toma)
print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
94


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [17]:
import random         #Para generar valores aleatorios

#Respuesta
def Get_Posibles_Hijos(Dias_Toma):
  Hijos = []
  long = len(Dias_Toma)
  for i in range(long):
    n = Dias_Toma.count(i)
    if n < 6 and n > 0:
      Hijos.append(i)
  Hijos.append(long)
  return Hijos

def Crear_Solucion(N):
  solucion = []
  for i in range(N):
    solucion.append(random.choice(Get_Posibles_Hijos(solucion)))
  return solucion

#Dia_por_Toma = [0, 1, 0, 2, 2]
Dia_por_Toma = []
print(Get_Posibles_Hijos(Dia_por_Toma))

Solucion = Crear_Solucion(30)
print(Solucion)
print(tuple(set(Solucion)))

[0]
[0, 1, 1, 0, 0, 1, 0, 0, 8, 1, 8, 0, 1, 1, 14, 15, 8, 15, 18, 8, 18, 8, 22, 22, 24, 8, 24, 27, 15, 24]
(0, 1, 8, 14, 15, 18, 22, 24, 27)


In [23]:
def Get_Posibles_Dias(Dias_Toma, nToma):
  Posibles = []
  dias = tuple(set(Dias_Toma))
  for d in dias:
    N = Dias_Toma.count(d)
    if N < 6 and d != Dias_Toma[nToma]:
      Posibles.append(d)
  return Posibles

Dia_por_Toma = [0, 1, 0, 2, 2, 0, 0, 0]
Posibles_dias = Get_Posibles_Dias(Dia_por_Toma, 1)
print(Posibles_dias)


[0, 2]


def Cambio_Dia(

In [61]:
def Cambio_Dia(Dias_Toma, nToma, nDia):
  # Cambia la toma 'nToma' al días 'nDia'
  if (nDia <= nToma):
    Dias_Toma[nToma] = nDia
  else:
    Dias_Toma[nToma] = nToma
    nP = -1
    for i in range(nToma+1, len(Dias_Toma)):
      if Dias_Toma[i] == nToma:
        if nP < 0:
          nP = i
        Dias_Toma[i] = nP

      if Dias_Toma[i] == nDia:
        Dias_Toma[i] = nToma


Dia_por_Toma = [0, 1, 2, 2, 0, 0, 0]

Cambio_Dia(Dia_por_Toma, 0, 3)
print(Dia_por_Toma)

Get_Coste(Get_Actores_Dia([0, 0, 0], Actores_por_Toma))

[0, 1, 2, 2, 4, 4, 4]


6

In [64]:
def genera_vecina(Dias_Toma, Actores_Toma):
  mejor_solucion = []
  mejor_coste = 10e100
  for i in range(len(Dias_Toma)):
    posibles = Get_Posibles_Dias(Dias_Toma, i)
    for dia in posibles:
      vecina = Dias_Toma.copy()
      Cambio_Dia(vecina, i, dia)
      vecina_coste = Get_Coste(Get_Actores_Dia(vecina, Actores_Toma))

      if vecina_coste <= mejor_coste:
        mejor_coste = vecina_coste
        mejor_solucion = vecina
  return mejor_solucion

Dia_por_Toma = Crear_Solucion(len(Actores_por_Toma))
print(Dia_por_Toma)
print("Coste solución inicial: ", Get_Coste(Get_Actores_Dia(Dia_por_Toma, Actores_por_Toma)))

nueva_solucion = genera_vecina(Dia_por_Toma, Actores_por_Toma)
print(nueva_solucion)
print("Coste solución vecina: ", Get_Coste(Get_Actores_Dia(nueva_solucion, Actores_por_Toma)))

nueva_solucion = genera_vecina(nueva_solucion, Actores_por_Toma)
print(nueva_solucion)
print("Coste solución vecina: ", Get_Coste(Get_Actores_Dia(nueva_solucion, Actores_por_Toma)))

[0, 0, 0, 0, 0, 5, 0, 5, 8, 5, 5, 8, 5, 8, 8, 8, 8, 5, 18, 19, 19, 21, 18, 21, 18, 21, 18, 19, 19, 19]
Coste solución inicial:  41
[0, 0, 0, 0, 0, 5, 0, 5, 8, 5, 5, 8, 5, 8, 8, 8, 8, 5, 18, 19, 19, 18, 18, 21, 18, 21, 18, 19, 19, 19]
Coste solución vecina:  39
[0, 0, 0, 0, 0, 5, 0, 5, 8, 5, 5, 8, 5, 8, 8, 8, 8, 5, 18, 19, 19, 18, 18, 18, 18, 18, 18, 19, 19, 19]
Coste solución vecina:  36
